In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../Data/requests.csv', index_col='unique_key')
df.head()

,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,status,community_board,borough,...,landmark,bbl,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,location,resolution_description,resolution_action_updated_date,waittime
unique_key,,,,,,,,,,,,,,,,,,,,,
67351762,2025-12-31 23:59:28,2026-01-01 00:40:32,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,11 MANHATTAN,MANHATTAN,...,EAST 104 STREET,1.016080e+09,998068.0,227879.0,40.792141,-73.950097,"{'type': 'Point', 'coordinates': [-73.95009707...",The New York City Police Department responded ...,2026-01-01T00:40:35.000,0.028519
67344624,2025-12-31 23:59:23,2026-01-01 01:03:42,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,09 MANHATTAN,MANHATTAN,...,WEST 144 STREET,1.020760e+09,998241.0,239901.0,40.825137,-73.949447,"{'type': 'Point', 'coordinates': [-73.94944723...",The New York City Police Department responded ...,2026-01-01T01:03:44.000,0.044664
67346873,2025-12-31 23:59:21,2026-01-01 00:58:13,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,Closed,18 BROOKLYN,BROOKLYN,...,EAST 56 STREET,3.078810e+09,1006135.0,165167.0,40.619995,-73.921167,"{'type': 'Point', 'coordinates': [-73.92116739...",The New York City Police Department responded ...,2026-01-01T00:58:18.000,0.040880
67353004,2025-12-31 23:59:20,2026-01-01 00:57:31,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,15 BROOKLYN,BROOKLYN,...,EAST 14 STREET,3.074150e+09,996476.0,154794.0,40.591542,-73.955979,"{'type': 'Point', 'coordinates': [-73.95597913...",The New York City Police Department responded ...,2026-01-01T00:57:34.000,0.040405
67350526,2025-12-31 23:59:12,2026-01-01 00:41:16,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,Closed,09 MANHATTAN,MANHATTAN,...,AMSTERDAM AVENUE,1.020570e+09,998220.0,238974.0,40.822593,-73.949525,"{'type': 'Point', 'coordinates': [-73.94952504...",The Police Department reviewed your complaint ...,2026-01-01T00:41:20.000,0.029213


In [18]:
print(f"{df.shape[0]} Rows, {df.shape[1]} Columns")
na_counts = df.isna().sum()
print(f"{'Column':<35} {'NA Counts':<12} {'NA %':<5}")
for col in na_counts.index:
    if na_counts[col] == 0: continue
    print(f"{col:<35} {na_counts[col]:<12} {na_counts[col]/df.shape[0]:.2%}")

600000 Rows, 31 Columns
Column                              NA Counts    NA % 
descriptor                          18335        3.06%
park_facility_name                  851          0.14%
incident_zip                        2960         0.49%
incident_address                    2951         0.49%
street_name                         2953         0.49%
cross_street_1                      4105         0.68%
cross_street_2                      3708         0.62%
intersection_street_1               4105         0.68%
intersection_street_2               3708         0.62%
city                                26818        4.47%
landmark                            26824        4.47%
bbl                                 47425        7.90%
x_coordinate_state_plane            2947         0.49%
y_coordinate_state_plane            2946         0.49%
latitude                            2947         0.49%
longitude                           2947         0.49%
location                            2947 

In [ ]:
#* Drop 
df = df[df['waittime'] >= 0]

count    68.000000
mean     -1.984571
std       0.010757
min      -1.999942
25%      -1.993825
50%      -1.986545
75%      -1.977063
max      -1.963287
Name: waittime, dtype: float64